In [1]:
import warnings
# warnings.filterwarnings("ignore")

In [7]:
import datetime
import glob
from functools import lru_cache, partial
from pprint import pprint
import ray

import gym_trading_env
import gymnasium as gym
import matplotlib.pyplot as plt
import MultiTrade
import numpy as np
import pandas as pd
import torch
from gym_trading_env.downloader import download, EXCHANGE_LIMIT_RATES
from gym_trading_env.environments import TradingEnv
from gym_trading_env.renderer import Renderer
from IPython.display import display
from ray import train, tune
from tqdm.autonotebook import tqdm
from utils.utils import build_dataset, build_market_image,preprocess_data,stack_arrays
from utils.forecast_utils import create_ts_preprocessor,create_ts_dataset

import datetime


In [12]:

EXCHANGE_LIMIT_RATES["binanceus"] = {
    "limit" : 200, # One request will query 200 data points (aka candlesticks).
    "pause_every" : 120, # it will pause every 120 requests.
    "pause" : 2, # the pause will last 2 seconds.
}

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


In [13]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT',
            'ETH/BTC','SOL/ETH',
            ]
target_pair='ETHUSDT'
time_frame="1h"

In [14]:

download(exchange_names = ["binanceus"],
    symbols= tqdm(COIN_PAIRS),
    timeframe= time_frame,
    dir = "data",
    since= datetime.datetime(year= 2022, month= 1, day=1),
)

  0%|          | 0/17 [00:00<?, ?it/s]

BTC/USDT downloaded from binanceus and stored at data/binanceus-BTCUSDT-1h.pkl
ETH/USDT downloaded from binanceus and stored at data/binanceus-ETHUSDT-1h.pkl
SOL/USDT downloaded from binanceus and stored at data/binanceus-SOLUSDT-1h.pkl
BNB/USDT downloaded from binanceus and stored at data/binanceus-BNBUSDT-1h.pkl
XRP/USDT downloaded from binanceus and stored at data/binanceus-XRPUSDT-1h.pkl
ADA/USDT downloaded from binanceus and stored at data/binanceus-ADAUSDT-1h.pkl
ETH/BTC downloaded from binanceus and stored at data/binanceus-ETHBTC-1h.pkl
SOL/ETH downloaded from binanceus and stored at data/binanceus-SOLETH-1h.pkl


BadSymbol: binanceus does not have market symbol BNB/ETH